In [1]:
import requests
import os
os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
import pandas as pd
import matplotlib.pyplot as plt
!pip install pandas_market_calendars
import requests
from datetime import datetime as dt
import pandas_market_calendars as mcal
from datetime import datetime as dt

In [2]:
# Create a calendar
nyse = mcal.get_calendar('NYSE')

In [24]:
timespan = nyse.schedule(start_date='2020-09-15', end_date='2021-03-15')

In [25]:
timespan.head()

,market_open,market_close
2020-09-15,2020-09-15 13:30:00+00:00,2020-09-15 20:00:00+00:00
2020-09-16,2020-09-16 13:30:00+00:00,2020-09-16 20:00:00+00:00
2020-09-17,2020-09-17 13:30:00+00:00,2020-09-17 20:00:00+00:00
2020-09-18,2020-09-18 13:30:00+00:00,2020-09-18 20:00:00+00:00
2020-09-21,2020-09-21 13:30:00+00:00,2020-09-21 20:00:00+00:00


In [26]:
times = list(timespan.index.values)

In [27]:
dates = []
for date in range(len(times)):
    ts = pd.to_datetime(str(times[date]))
    d = ts.strftime('%Y-%m-%d')
    dates.append(d)

In [28]:
dates

['2020-09-15',
 '2020-09-16',
 '2020-09-17',
 '2020-09-18',
 '2020-09-21',
 '2020-09-22',
 '2020-09-23',
 '2020-09-24',
 '2020-09-25',
 '2020-09-28',
 '2020-09-29',
 '2020-09-30',
 '2020-10-01',
 '2020-10-02',
 '2020-10-05',
 '2020-10-06',
 '2020-10-07',
 '2020-10-08',
 '2020-10-09',
 '2020-10-12',
 '2020-10-13',
 '2020-10-14',
 '2020-10-15',
 '2020-10-16',
 '2020-10-19',
 '2020-10-20',
 '2020-10-21',
 '2020-10-22',
 '2020-10-23',
 '2020-10-26',
 '2020-10-27',
 '2020-10-28',
 '2020-10-29',
 '2020-10-30',
 '2020-11-02',
 '2020-11-03',
 '2020-11-04',
 '2020-11-05',
 '2020-11-06',
 '2020-11-09',
 '2020-11-10',
 '2020-11-11',
 '2020-11-12',
 '2020-11-13',
 '2020-11-16',
 '2020-11-17',
 '2020-11-18',
 '2020-11-19',
 '2020-11-20',
 '2020-11-23',
 '2020-11-24',
 '2020-11-25',
 '2020-11-27',
 '2020-11-30',
 '2020-12-01',
 '2020-12-02',
 '2020-12-03',
 '2020-12-04',
 '2020-12-07',
 '2020-12-08',
 '2020-12-09',
 '2020-12-10',
 '2020-12-11',
 '2020-12-14',
 '2020-12-15',
 '2020-12-16',
 '2020-12-

In [8]:
ticker = 'NVDA'
range_agg = '/range/1/minute/'
unadjusted = 'unadjusted=true'
sort = '&sort=asc'
limit = '&limit=50000'
POLYGON_URL = 'https://api.polygon.io/v2/aggs/ticker/'

In [9]:
import data_config
datetime = []
open_price = []
high_price = []
low_price = []
close_price = []
volume = []
for date in dates:
    polygon = POLYGON_URL + ticker + range_agg + date + '/' + date + '?' + unadjusted + sort + limit + data_config.POLYGON_API
    r_polygon = requests.get(polygon)
    polygon_response = r_polygon.json()
    for aggs in range(len(polygon_response['results'])):
        datetime.append(polygon_response['results'][aggs]['t'])
        open_price.append(polygon_response['results'][aggs]['o'])
        high_price.append(polygon_response['results'][aggs]['h'])
        low_price.append(polygon_response['results'][aggs]['l'])
        close_price.append(polygon_response['results'][aggs]['c'])
        volume.append(polygon_response['results'][aggs]['v'])

In [10]:
df_ticks = pd.DataFrame(list(zip(datetime,open_price,high_price,low_price, close_price,volume)),
columns =['datetime', 'open', 'high', 'low', 'close', 'volume'])

In [11]:
df_ticks.head()

,datetime,open,high,low,close,volume
0,1600156800000,519.00,520.00,519.00,520.00,1379
1,1600156860000,521.00,521.00,521.00,521.00,643
2,1600157400000,520.93,520.93,520.93,520.93,206
3,1600157700000,520.25,520.25,520.25,520.25,346
4,1600157880000,520.00,520.00,520.00,520.00,231


In [12]:
len(df_ticks)

68637

In [13]:
df_ticks.to_csv('stock_prices__min_test_NVDA.csv',sep = ",", index=True)

In [14]:
len(dates)

125

In [42]:
ticker = 'X:DOGEUSD'
range_agg = '/range/1/minute/'
unadjusted = 'unadjusted=true'
sort = '&sort=asc'
limit = '&limit=50000'
POLYGON_URL = 'https://api.polygon.io/v2/aggs/ticker/'

"""https://api.polygon.io/v2/aggs/ticker/X:BTCUSD/range/1/minute/2020-10-14/2020-10-14?unadjusted=true&sort=asc&limit=50000&apiKey=x6XqEaDjOWJXwVQRyBQ5kMEi8KlCYZqo"""

'https://api.polygon.io/v2/aggs/ticker/X:BTCUSD/range/1/minute/2020-10-14/2020-10-14?unadjusted=true&sort=asc&limit=50000&apiKey=x6XqEaDjOWJXwVQRyBQ5kMEi8KlCYZqo'

In [43]:
import data_config
datetime = []
open_price = []
high_price = []
low_price = []
close_price = []
volume = []
for date in dates:
    polygon = POLYGON_URL + ticker + range_agg + date + '/' + date + '?' + unadjusted + sort + limit + data_config.POLYGON_API
    r_polygon = requests.get(polygon)
    polygon_response = r_polygon.json()
    for aggs in range(len(polygon_response['results'])):
        datetime.append(polygon_response['results'][aggs]['t'])
        open_price.append(polygon_response['results'][aggs]['o'])
        high_price.append(polygon_response['results'][aggs]['h'])
        low_price.append(polygon_response['results'][aggs]['l'])
        close_price.append(polygon_response['results'][aggs]['c'])
        volume.append(polygon_response['results'][aggs]['v'])

In [44]:
df_ticks = pd.DataFrame(list(zip(datetime,open_price,high_price,low_price, close_price,volume)),
columns =['datetime', 'open', 'high', 'low', 'close', 'volume'])

In [45]:
df_ticks.head()

,datetime,open,high,low,close,volume
0,1600128060000,0.002795,0.002801,0.002786,0.002786,120.0
1,1600128120000,0.002805,0.002805,0.002799,0.002799,50.0
2,1600128180000,0.002800,0.002804,0.002797,0.002800,6330.0
3,1600128240000,0.002798,0.002806,0.002798,0.002802,8690.0
4,1600128300000,0.002799,0.002804,0.002798,0.002802,23300.0


In [46]:
len(df_ticks)

144063

In [47]:
df_ticks.to_csv('cryto_prices__min_test_DOGECOIN.csv',sep = ",", index=True)

In [48]:
len(dates)

125